In [8]:
import numpy as np
import pandas as pd
import seaborn as sns

from urllib.request import urlopen
from bs4 import BeautifulSoup

**Dans le dataset initial (5000_movies.csv)**, il y a beaucoup de valeurs manquantes dans `gross` et dans `budget` donc imputation délicate mais comme on veut garder ces variables, on va alors supprimer les lignes. Avant cette suppression on peut essayer de scraper IMDB pour obtenir des informations supplémentaires. Pour cela, un petit notebook séparé dans le même dossier permet de récupérer les données supplémentaires éventuelles, de les ajouter dans les colonnes `gross` et `budget` du dataframe `data`. Comme l'éxecution est un peu longue, on va sauvegarder ce dataframe dans un nouveau csv pour ne pas avoir à le refaire à chaque fois.

In [61]:
data = pd.read_csv('5000_movies.csv')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   color                   5024 non-null   object 
 1   director_name           4939 non-null   object 
 2   num_critic_for_reviews  4993 non-null   float64
 3   duration                5028 non-null   float64
 4   director_fb_likes       4939 non-null   float64
 5   actor_3_fb_likes        5020 non-null   float64
 6   actor_2_name            5030 non-null   object 
 7   actor_1_fb_likes        5036 non-null   float64
 8   gross                   4159 non-null   float64
 9   genres                  5043 non-null   object 
 10  actor_1_name            5036 non-null   object 
 11  movie_title             5043 non-null   object 
 12  num_voted_users         5043 non-null   int64  
 13  cast_total_fb_likes     5043 non-null   int64  
 14  actor_3_name            5020 non-null   

In [60]:
data.isna().sum()

color                      19
director_name             104
num_critic_for_reviews     50
duration                   15
director_fb_likes         104
actor_3_fb_likes           23
actor_2_name               13
actor_1_fb_likes            7
gross                     884
genres                      0
actor_1_name                7
movie_title                 0
num_voted_users             0
cast_total_fb_likes         0
actor_3_name               23
facenumber_in_poster       13
plot_keywords             153
movie_imdb_link             0
num_user_for_reviews       21
language                   12
country                     5
content_rating            303
budget                    492
title_year                108
actor_2_fb_likes           13
imdb_score                  0
aspect_ratio              329
movie_fb_likes              0
dtype: int64

In [3]:
# scraping des données 'gross' et 'budget'
def scrap_gross_budget(row):
    link = row['movie_imdb_link']
    
    # si une des valeurs est manquante on va tenter d'aller les chercher
    if (np.isnan(row['gross'])) | (np.isnan(row['budget'])) :
        try :
            # on récupère la page web
            page = urlopen(link).read()
            soup = BeautifulSoup(page)

            # on récupère dans la page le Domestic Gross
            scrap_gross = soup.find('li', {'data-testid':'title-boxoffice-grossdomestic'}).find('span', {'class':"ipc-metadata-list-item__list-content-item"})
            scrap_gross = float(''.join(c for c in scrap_gross.contents[0] if c.isdigit()))

            # on récupère dans la page le Budget
            scrap_budget = soup.find('li', {'data-testid':'title-boxoffice-budget'}).find('span', {'class':"ipc-metadata-list-item__list-content-item"})
            scrap_budget = float(''.join(c for c in scrap_budget.contents[0] if c.isdigit()))
                    
        except AttributeError as err:
            return row[['gross', 'budget']]
    
    if (np.isnan(row['gross'])) & (np.isnan(row['budget'])) :
        return scrap_gross, scrap_budget
    
    elif (np.isnan(row['gross'])) :
        return scrap_gross, row['budget']
    
    elif (np.isnan(row['budget'])) :
        return row['gross'], scrap_budget
    
    else:
        return row[['gross', 'budget']]

In [4]:
from tqdm import tqdm
tqdm.pandas()
data[['gross', 'budget']] = data.progress_apply(scrap_gross_budget, axis=1)

100%|███████████████████████████████████████| 5043/5043 [54:46<00:00,  1.53it/s]


In [5]:
# enregistrement du dataframe en csv
data.to_csv('5000_movies_bis.csv', index=False)